<a href="https://colab.research.google.com/github/ravindrabharathi/Project1/blob/master/EIP4/session5/Crop_images_PersonAttributes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code to crop away black pixels used to pad the person database images 

Mount google drive and copy project data

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
project_folder="/content/gdrive/My Drive/EIP4/session5"

!rm -R ./resized
!rm hvc_annotations.csv
!unzip -q "/content/gdrive/My Drive/EIP4/session5/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
rm: cannot remove './resized': No such file or directory
rm: cannot remove 'hvc_annotations.csv': No such file or directory
gdrive/  hvc_annotations.csv  resized/  sample_data/


Load necessary libraries 

In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16,ResNet50V2
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input,Conv2D,GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(seed=2019)


Using TensorFlow backend.


Function to drop black padding pixels from images 

In [0]:
# from https://codereview.stackexchange.com/questions/132914/crop-black-border-of-image-using-numpy
def crop_image(img,tol=0):
  # img is 2D image data
  # tol  is tolerance
  channels=[]
  min_width=224
  min_height=224
  for i in range(3):
    img1=img[:,:,i]
    #print(img1.shape)
    mask = img1>tol
    img2=img1[np.ix_(mask.any(1),mask.any(0))]
    if img2.shape[1]<min_width:
      min_width=img2.shape[1]
    if img2.shape[0]<min_height:
      min_height=img2.shape[0]  
    channels.append(img2)
    #print(img2.shape,min_width)
    #print(min_width)
  #
    #print(type(channels[i]))
  for i in range(3):
    img4=channels[i]
    img4=img4[0:min_height,0:min_width]
    channels[i]=img4
  img3=np.stack([channels[0], channels[1], channels[2]],axis=-1)
  #print(img3.shape)
  return img3 

Process all images in person database 

In [0]:
import os
!rm -R './cropped/'
!mkdir './cropped'
print('created cropped dir...')
image_list=os.listdir('./resized/')
print('starting image cropping ...')
for i in range(len(image_list)):
  #print('./resized/'+image_list[i])
  image=cv2.imread('./resized/'+image_list[i])
  #cv2_imshow(image)
  image1=crop_image(image,tol=10)
  cv2.imwrite('./cropped/'+image_list[i],image1)
  if (i%1000==0):
    print('processed ',i,'images')

created cropped dir...
starting image cropping ...
processed  0 images
processed  1000 images
processed  2000 images
processed  3000 images
processed  4000 images
processed  5000 images
processed  6000 images
processed  7000 images
processed  8000 images
processed  9000 images
processed  10000 images
processed  11000 images
processed  12000 images
processed  13000 images


Archive(tar) the resulting images and copy them as cropped.tar to the google drive project directory 

In [0]:

image_list2=os.listdir('./cropped/')
print(len(image_list),len(image_list2))

13573 13573


In [0]:
!tar -cvf cropped.tar './cropped/'

In [0]:
!echo $project_folder

/content/gdrive/My Drive/EIP4/session5


In [0]:
!cp './cropped.tar' '$project_folder'

### Now that we have copied the cropped images to our project directory , we can use this dataset to train the model for multi output classification of age ,gender ,pose , weight , etc